In [1]:
import pandas as pd
import biogeme.logging as blog
import biogeme.biogeme as bio
from biogeme import models
from biogeme.results import calculate_correlation
from biogeme.expressions import Beta

import numpy as np
import biogeme.database as db
import biogeme.optimization as opt
from biogeme.expressions import Beta, DefineVariable
import biogeme.expressions as exp


intermodal_data = pd.read_csv('/Users/lei/Documents/毕业论文/jiadingsurvey/intermodalAnalysis/2.fromRPtoSP/fitmodal_data/allmodal_data_travelinfo_social_wide.csv')

intermodal_data = intermodal_data[intermodal_data['totaldistance_car'] < 50000]
intermodal_data['totalcost_car_new'] = (intermodal_data['totaldistance_car']/1000) * 1.09
database = db.Database("intermodal_data", intermodal_data)
globals().update(database.variables)




In [3]:
totalcost_metro_car_new = database.DefineVariable('totalcost_metro_car_new',totalcost_metro_car-12)


In [2]:
ASC_bus_metro   = exp.Beta('ASC_bus_metro',0,None ,None ,0)  # 常数，相当于`$\beta{car}$`
ASC_metro_car = exp.Beta('ASC_metro_car',0,None ,None ,0)
ASC_bike_metro = exp.Beta('ASC_bike_metro',0,None ,None ,0)
ASC_ebike_metro = exp.Beta('ASC_ebike_metro',0,None ,None ,0)
ASC_metro = exp.Beta('ASC_metro',0,None ,None ,0)
ASC_bus = exp.Beta('ASC_bus',0,None ,None ,0)
ASC_car = exp.Beta('ASC_car',0,None ,None ,1)

B_TIME_bus_metro    = exp.Beta('B_TIME_bus_metro',0,None ,None ,0)  # bus_metro 出行时长前面的系数
B_TIME_metro_car = exp.Beta('B_TIME_metro_car',0,None ,None ,0)  # metro_car 出行时长前面的系数
B_TIME_bike_metro = exp.Beta('B_TIME_bike_metro',0,None ,None ,0)  # metro 出行时长前面的系数
B_TIME_ebike_metro = exp.Beta('B_TIME_ebike_metro',0,None ,None ,0)  # metro 出行时长前面的系数
B_TIME_metro = exp.Beta('B_TIME_metro',0,None ,None ,0)  # metro 出行时长前面的系数
B_TIME_bus = exp.Beta('B_TIME_bus',0,None ,None ,0)  # bus 出行时长前面的系数
B_TIME_car = exp.Beta('B_TIME_car',0,None ,None ,0)  # car 出行时长前面的系数

B_COST_Transit = exp.Beta('B_COST_Transit',0,None ,None ,0)  # 出行费用前面的系数
B_COST_car = exp.Beta('B_COST_car',0,None ,None ,0)  # 出行费用前面的系数
# B_COST_metro_car = exp.Beta('B_COST_metro_car',0,None ,None ,0)  # 出行费用前面的系数


B_ACTIME_walk = exp.Beta('B_ACTIME_walk',0,None ,None ,0)  # 换乘时间前面的系数
B_ACTIME_bike_ebike = exp.Beta('B_ACTIME_bike_ebike',0,None ,None ,0)
B_ACTIME_car = exp.Beta('B_ACTIME_car',0,None ,None ,0)


B_TRNUMS= exp.Beta('B_TRNUMS',0,None ,None ,0)  # 换乘次数前面的系数
# B_TRNUMS_bus = exp.Beta('B_TRNUMS_bus',0,None ,None ,0)  # 换乘次数前面的系数
# B_TRNUMS_bus_metro = exp.Beta('B_TRNUMS_bus_metro',0,None ,None ,0)  # 换乘次数前面的系数
B_TRNUMS_car = exp.Beta('B_TRNUMS_car',0,None ,None ,1)  # 换乘次数前面的系数

# MU = Beta('MU_metro', 1, 1, 10, 0)
# MU = Beta('MU_metro', 1, 1, 10, 0)

# Definition of the utility functions




V1 = ASC_bus_metro + \
     B_TIME_bus_metro * totalduration_bus_metro + \
     B_COST_Transit * totalcost_bus_metro + \
     B_ACTIME_walk * ac_eg_transfer_duration_bus_metro + \
     B_TRNUMS * transfernum_bus_metro

V2 = ASC_metro_car + \
     B_TIME_metro_car * totalduration_metro_car + \
     B_COST_Transit * totalcost_metro_car + \
     B_ACTIME_car * ac_eg_transfer_duration_metro_car + \
     B_TRNUMS * transfernum_metro_car
V3 = ASC_bike_metro + \
     B_TIME_bike_metro * totalduration_bike_metro + \
     B_COST_Transit * totalcost_bike_metro + \
     B_ACTIME_bike_ebike * ac_eg_transfer_duration_bike_metro + \
     B_TRNUMS * transfernum_bike_metro
V4 = ASC_ebike_metro + \
     B_TIME_ebike_metro * totalduration_ebike_metro + \
     B_COST_Transit * totalcost_ebike_metro + \
     B_ACTIME_bike_ebike * ac_eg_transfer_duration_ebike_metro + \
     B_TRNUMS * transfernum_ebike_metro
V5 = ASC_metro + \
     B_TIME_metro * totalduration_metro + \
     B_COST_Transit * totalcost_metro + \
     B_ACTIME_walk * ac_eg_transfer_duration_metro + \
     B_TRNUMS * transfernum_metro
V6 = ASC_bus + \
     B_TIME_bus * totalduration_bus + \
     B_COST_Transit * totalcost_bus + \
     B_ACTIME_walk * ac_eg_transfer_duration_bus + \
     B_TRNUMS * transfernum_bus
V7 = ASC_car + \
     B_TIME_car * totalduration_car + \
     B_COST_car * totalcost_car_new+ \
     B_TRNUMS_car * transfernum_car


V = {1: V1,
     2: V2,
     3: V3,
     4: V4,
     5: V5,
     6: V6,
     7: V7}

av = {1: bus_metro_av,
      2: metro_car_av,
      3: bike_metro_av,
      4: ebike_metro_av,
      5: metro_av,
      6: bus_av,
      7: car_av}





In [5]:
MU_inter = Beta('MU_inter', 1, 1, None, 0)
MU_single = Beta('MU_single', 1, 1, None, 1)

metro_nest = MU_inter, [1, 2, 3, 4]
bus_nest = MU_single, [5,6,7]


nests = bus_nest,metro_nest


nestedlogprob = models.lognested (V, None , nests , choice )# 用log likelihood 来标定模型

nested_biogeme  = bio.BIOGEME(database, nestedlogprob)
nested_biogeme.modelName = "nested_logit_inter_single"

nested_biogeme.generateHtml = True
nested_biogeme.generatePickle = False

nested_results = nested_biogeme.estimate()

print(f"HTML file:    {nested_results.data.htmlFileName}")
print(f"Pickle file:  {nested_results.data.pickleFileName }")

Obsolete syntax. Use generate_html instead of generateHtml
Obsolete syntax. Use generate_pickle instead of generatePickle


HTML file:    nested_logit_inter_single~03.html
Pickle file:  None


In [6]:

ml_logprob = models.loglogit(V, None, choice) # 用log likelihood 来标定模型

ml_biogeme  = bio.BIOGEME(database, ml_logprob)
ml_biogeme.modelName = "intermoal_only_travelinfo_advance_model3"

ml_biogeme.generateHtml = True
ml_biogeme.generatePickle = False

ml_results = ml_biogeme.estimate()

print(f"HTML file:    {ml_results.data.htmlFileName}")
print(f"Pickle file:  {ml_results.data.pickleFileName }")


Obsolete syntax. Use generate_html instead of generateHtml
Obsolete syntax. Use generate_pickle instead of generatePickle


HTML file:    intermoal_only_travelinfo_advance_model3.html
Pickle file:  None


In [36]:
nested_results.getBetaValues().keys()

dict_keys(['ASC_bike_metro', 'ASC_bus', 'ASC_bus_metro', 'ASC_ebike_metro', 'ASC_metro', 'ASC_metro_car', 'B_ACTIME_bike_ebike', 'B_ACTIME_car', 'B_ACTIME_walk', 'B_COST_Transit', 'B_COST_car', 'B_TIME_bike_metro', 'B_TIME_bus', 'B_TIME_bus_metro', 'B_TIME_car', 'B_TIME_ebike_metro', 'B_TIME_metro', 'B_TIME_metro_car', 'B_TRNUMS', 'MU_inter'])

In [21]:
from scipy.stats import chi2
diff = nested_results.getEstimatedParameters()['Value'] - ml_results.getEstimatedParameters()['Value']
variance_diff = nested_results.getCorrelationResults()['Covariance'] + ml_results.getCorrelationResults()['Covariance']

# Compute the Hausman test statistic
hausman_test_statistic = diff @ np.linalg.inv(variance_diff) @ diff.T

# Compare the test statistic to a chi-squared distribution
degrees_of_freedom = len(diff)
p_value = 1.0 - chi2.cdf(hausman_test_statistic, degrees_of_freedom)

print(f'Hausman Test Statistic: {hausman_test_statistic}')
print(f'Degrees of Freedom: {degrees_of_freedom}')
print(f'P-value: {p_value}')

LinAlgError: 1-dimensional array given. Array must be at least two-dimensional

In [19]:
variance_diff

ASC_bus-ASC_bike_metro            0.052638
ASC_bus_metro-ASC_bike_metro      0.136837
ASC_bus_metro-ASC_bus             0.053680
ASC_ebike_metro-ASC_bike_metro    0.127017
ASC_ebike_metro-ASC_bus           0.046233
                                    ...   
MU_inter-B_TIME_car                    NaN
MU_inter-B_TIME_ebike_metro            NaN
MU_inter-B_TIME_metro                  NaN
MU_inter-B_TIME_metro_car              NaN
MU_inter-B_TRNUMS                      NaN
Name: Covariance, Length: 190, dtype: float64

In [24]:
import numpy as np

# Generated data
np.random.seed(42)
n_obs = 100

# Independent variable X with a constant term
X = np.column_stack((np.ones(n_obs), np.random.rand(n_obs, 1)))

# True coefficients for the linear regression
true_beta0 = 2.0
true_beta1 = 3.0

# Dependent variable Y with random noise
Y = true_beta0 + true_beta1 * X[:, 1] + np.random.normal(0, 1, size=n_obs)

# Assume you have the fixed effects estimate beta_FE
beta_FE = np.array([true_beta0, true_beta1])

# 1. Calculate residuals
residuals_FE = Y - np.dot(X, beta_FE)

# 2. Compute error variance
sigma_squared_FE = np.sum(residuals_FE ** 2) / (n_obs - len(beta_FE))

# 3. Compute variance-covariance matrix
var_FE = sigma_squared_FE * np.linalg.inv(np.dot(X.T, X))

print("Variance-Covariance Matrix for Fixed Effects:")
print(var_FE)


Variance-Covariance Matrix for Fixed Effects:
[[ 0.02966342 -0.04518255]
 [-0.04518255  0.09609612]]


In [ ]:
def perform_mlogit():
    pass
def perform_nl():
    pass
def compare_mlogit_vs_nl():
    pass
